# Datos de edades y sexos por secciones electorales

En este cuaderno transformamos el dataframe de las distribuciones de edades y sexos por secciones electorales que obtuvimos mediante el cuaderno en R, que tomo el fichero de formato .px del INE y generó el df.

Obtendremos otro dataframe que nos será fácil integrar con los resultados electorales por secciones, así como otros datos sociodemográficos.

Comenzamos cargando las librerías necesarias y el dataframe original. Hemos tomado el correspondiente al año 2012, que integraremos en el dataframe de las elecciones de noviembre de 2011.



In [42]:
import pandas as pd
import numpy as np

In [43]:
import boto3

BUCKET_NAME = 'electomedia' 

# sustituir por credenciales de acceso. 
s3 = boto3.resource('s3', aws_access_key_id = 'XXXXXXX', 
                          aws_secret_access_key= 'XXXXXXX')

In [44]:
import botocore.exceptions

KEY = 'INE/2012_df_edades.csv' 

try:
    s3.Bucket(BUCKET_NAME).download_file(KEY, '2012_df_edades.csv')
      
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [5]:
edades = pd.read_csv('2012_df_edades.csv', encoding='latin1', dtype = {'sección' : 'str'})

Echamos una primera vista, y parece que tiene unos nombres algo extraños en sus columnas. Las cambiamos pues. Se trata de un dataset amplio, con más de 2 millones de filas.

In [6]:
edades.head()

,Unnamed: 0,edad..grupos.quinquenales.,sección,sexo,value
0,1,Total,TOTAL,Ambos Sexos,47265321
1,2,0-4,TOTAL,Ambos Sexos,2443703
2,3,5-9,TOTAL,Ambos Sexos,2404312
3,4,10-14,TOTAL,Ambos Sexos,2214708
4,5,15-19,TOTAL,Ambos Sexos,2227550


In [7]:
edades.shape

(2374614, 5)

In [8]:
edades.rename(columns={'edad..grupos.quinquenales.' : 'grupos_quinquenales' }, inplace = True)

Nos interesan solo los datos de las secciones electorales, no los agregados por provincias o el total nacional, por lo que que elimnamos los totales.

In [9]:
edades_aj = edades.loc[edades['sección'] != 'TOTAL']

In [10]:
edades_aj

,Unnamed: 0,grupos_quinquenales,sección,sexo,value
22,23,Total,0100101001,Ambos Sexos,1341
23,24,0-4,0100101001,Ambos Sexos,119
24,25,5-9,0100101001,Ambos Sexos,103
25,26,10-14,0100101001,Ambos Sexos,65
26,27,15-19,0100101001,Ambos Sexos,47
...,...,...,...,...,...
2374609,2374610,80-84,5200108014,Mujeres,3
2374610,2374611,85-89,5200108014,Mujeres,4
2374611,2374612,90-94,5200108014,Mujeres,2
2374612,2374613,95-99,5200108014,Mujeres,0


In [11]:
edades_aj.columns

Index(['Unnamed: 0', 'grupos_quinquenales', 'sección', 'sexo', 'value'], dtype='object')

Parece que tenemos un doble índice por lo que eliminamos la columna redundante.

In [12]:
edades_aj.drop(['Unnamed: 0'], axis = 1)

,grupos_quinquenales,sección,sexo,value
22,Total,0100101001,Ambos Sexos,1341
23,0-4,0100101001,Ambos Sexos,119
24,5-9,0100101001,Ambos Sexos,103
25,10-14,0100101001,Ambos Sexos,65
26,15-19,0100101001,Ambos Sexos,47
...,...,...,...,...
2374609,80-84,5200108014,Mujeres,3
2374610,85-89,5200108014,Mujeres,4
2374611,90-94,5200108014,Mujeres,2
2374612,95-99,5200108014,Mujeres,0


Ahora nos centramos en la distribución por edades, por lo que nos quedamos con los registros de incluyen ambos sexos.

In [13]:
edades_aj = edades_aj.loc[edades_aj['sexo'] == 'Ambos Sexos']

In [14]:
edades_aj

,Unnamed: 0,grupos_quinquenales,sección,sexo,value
22,23,Total,0100101001,Ambos Sexos,1341
23,24,0-4,0100101001,Ambos Sexos,119
24,25,5-9,0100101001,Ambos Sexos,103
25,26,10-14,0100101001,Ambos Sexos,65
26,27,15-19,0100101001,Ambos Sexos,47
...,...,...,...,...,...
791533,791534,80-84,5200108014,Ambos Sexos,8
791534,791535,85-89,5200108014,Ambos Sexos,8
791535,791536,90-94,5200108014,Ambos Sexos,3
791536,791537,95-99,5200108014,Ambos Sexos,0


Efectuamos el pivot para poner como columna los rangos de edades. Vemos que el orden de las edades no queda de forma correcta con lo que variamos el orden.

In [16]:
edades_pivot = edades_aj.pivot(index = 'sección', columns = 'grupos_quinquenales', values = 'value')

In [17]:
edades_pivot

grupos_quinquenales,0-4,10-14,100 y más,15-19,20-24,25-29,30-34,35-39,40-44,45-49,...,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,Total
sección,,,,,,,,,,,,,,,,,,,,,
0100101001,119,65,0,47,45,68,117,215,154,113,...,64,45,35,30,26,13,4,7,1,1341
0100101002,131,89,0,58,59,75,152,201,176,132,...,56,52,50,32,27,22,10,6,0,1528
0100201001,91,81,0,86,104,137,128,118,130,174,...,161,146,95,68,79,71,42,15,1,1979
0100201002,64,52,0,75,97,110,114,98,105,155,...,163,136,109,52,70,49,31,9,5,1702
0100201003,46,41,0,45,45,80,78,76,69,87,...,66,73,60,41,31,43,23,12,0,1042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5200108010,74,84,0,80,115,129,120,92,93,101,...,108,71,62,49,42,37,11,2,0,1480
5200108011,148,148,0,169,162,144,147,140,167,167,...,134,90,53,33,36,19,13,4,0,2107
5200108012,143,125,0,118,129,196,249,215,219,224,...,151,108,94,75,70,45,30,4,1,2532


In [18]:
edades_pivot.columns

Index(['0-4', '10-14', '100 y más', '15-19', '20-24', '25-29', '30-34',
       '35-39', '40-44', '45-49', '5-9', '50-54', '55-59', '60-64', '65-69',
       '70-74', '75-79', '80-84', '85-89', '90-94', '95-99', 'Total'],
      dtype='object', name='grupos_quinquenales')

In [19]:
orden_cols = ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34',
       '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69',
       '70-74', '75-79', '80-84', '85-89', '90-94', '95-99', '100 y más', 'Total']

In [20]:
edades_pivot = edades_pivot[orden_cols]

Ya tenemos el orden correcto.

In [21]:
edades_pivot

grupos_quinquenales,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,...,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100 y más,Total
sección,,,,,,,,,,,,,,,,,,,,,
0100101001,119,103,65,47,45,68,117,215,154,113,...,45,35,30,26,13,4,7,1,0,1341
0100101002,131,120,89,58,59,75,152,201,176,132,...,52,50,32,27,22,10,6,0,0,1528
0100201001,91,60,81,86,104,137,128,118,130,174,...,146,95,68,79,71,42,15,1,0,1979
0100201002,64,44,52,75,97,110,114,98,105,155,...,136,109,52,70,49,31,9,5,0,1702
0100201003,46,43,41,45,45,80,78,76,69,87,...,73,60,41,31,43,23,12,0,0,1042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5200108010,74,60,84,80,115,129,120,92,93,101,...,71,62,49,42,37,11,2,0,0,1480
5200108011,148,155,148,169,162,144,147,140,167,167,...,90,53,33,36,19,13,4,0,0,2107
5200108012,143,145,125,118,129,196,249,215,219,224,...,108,94,75,70,45,30,4,1,0,2532


Vamos ahora con los datos por sexos, pero que acumulen todas las edades. Cargamos el dataset original de nuevo, y borramos la columna redundante del índice.

In [22]:
edades

,Unnamed: 0,grupos_quinquenales,sección,sexo,value
0,1,Total,TOTAL,Ambos Sexos,47265321
1,2,0-4,TOTAL,Ambos Sexos,2443703
2,3,5-9,TOTAL,Ambos Sexos,2404312
3,4,10-14,TOTAL,Ambos Sexos,2214708
4,5,15-19,TOTAL,Ambos Sexos,2227550
...,...,...,...,...,...
2374609,2374610,80-84,5200108014,Mujeres,3
2374610,2374611,85-89,5200108014,Mujeres,4
2374611,2374612,90-94,5200108014,Mujeres,2
2374612,2374613,95-99,5200108014,Mujeres,0


In [23]:
edades.drop(['Unnamed: 0'], axis = 1)

,grupos_quinquenales,sección,sexo,value
0,Total,TOTAL,Ambos Sexos,47265321
1,0-4,TOTAL,Ambos Sexos,2443703
2,5-9,TOTAL,Ambos Sexos,2404312
3,10-14,TOTAL,Ambos Sexos,2214708
4,15-19,TOTAL,Ambos Sexos,2227550
...,...,...,...,...
2374609,80-84,5200108014,Mujeres,3
2374610,85-89,5200108014,Mujeres,4
2374611,90-94,5200108014,Mujeres,2
2374612,95-99,5200108014,Mujeres,0


Eliminamos los registros de los totales por provincia y demás, quedándonos solo con las secciones, igual que antes. Por otro lado, por grupos quinquenales de edad SÍ queremos los totales, por lo que nos quedamos con ellos.

In [24]:
edades_sexos = edades.loc[edades['sección'] != 'TOTAL']

In [25]:
edades_sexos = edades_sexos.loc[edades_sexos['grupos_quinquenales'] == 'Total']

In [26]:
edades_sexos.columns

Index(['Unnamed: 0', 'grupos_quinquenales', 'sección', 'sexo', 'value'], dtype='object')

Una vez que tenemos los totales de los grupos de edad esta columna ya no nos aporta nada, por lo que la eliminamos.

In [27]:
edades_sexos.drop(['Unnamed: 0', 'grupos_quinquenales'], axis = 1, inplace = True)

In [28]:
edades_sexos = edades_sexos.loc[edades_sexos['sección'] != 'TOTAL']

In [29]:
edades_sexos

,sección,sexo,value
22,0100101001,Ambos Sexos,1341
44,0100101002,Ambos Sexos,1528
66,0100201001,Ambos Sexos,1979
88,0100201002,Ambos Sexos,1702
110,0100201003,Ambos Sexos,1042
...,...,...,...
2374504,5200108010,Mujeres,745
2374526,5200108011,Mujeres,1050
2374548,5200108012,Mujeres,1225
2374570,5200108013,Mujeres,788


Ahora hacemos el pivot para poner los datos de hombres y mujeres como columnas. No nos interesa la de ambos sexos.

In [31]:
edades_sexos_pivot = edades_sexos.pivot(index = 'sección', columns = 'sexo', values = 'value')

In [32]:
edades_sexos_pivot = edades_sexos_pivot[['Hombres', 'Mujeres']]

In [33]:
edades_sexos_pivot

sexo,Hombres,Mujeres
sección,,
0100101001,698,643
0100101002,786,742
0100201001,1012,967
0100201002,894,808
0100201003,517,525
...,...,...
5200108010,735,745
5200108011,1057,1050
5200108012,1307,1225


Ahora hacemos un merge de los dos datsets, el de edades y sexos, tomando como eje la sección.

In [34]:
edades_result = pd.merge(edades_pivot, edades_sexos_pivot, how="outer", on=['sección'])

In [35]:
edades_result

,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,...,70-74,75-79,80-84,85-89,90-94,95-99,100 y más,Total,Hombres,Mujeres
sección,,,,,,,,,,,,,,,,,,,,,
0100101001,119,103,65,47,45,68,117,215,154,113,...,30,26,13,4,7,1,0,1341,698,643
0100101002,131,120,89,58,59,75,152,201,176,132,...,32,27,22,10,6,0,0,1528,786,742
0100201001,91,60,81,86,104,137,128,118,130,174,...,68,79,71,42,15,1,0,1979,1012,967
0100201002,64,44,52,75,97,110,114,98,105,155,...,52,70,49,31,9,5,0,1702,894,808
0100201003,46,43,41,45,45,80,78,76,69,87,...,41,31,43,23,12,0,0,1042,517,525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5200108010,74,60,84,80,115,129,120,92,93,101,...,49,42,37,11,2,0,0,1480,735,745
5200108011,148,155,148,169,162,144,147,140,167,167,...,33,36,19,13,4,0,0,2107,1057,1050
5200108012,143,145,125,118,129,196,249,215,219,224,...,75,70,45,30,4,1,0,2532,1307,1225


Hacemos un reset del índice, que hasta ahora era la sección, y cuya columna nombramos 'cod_sec' que nos servirá cuando hagamos la unificacion con los resultados de las elecciones y otros datos por secciones.

In [36]:
edades_result = edades_result.reset_index()

In [37]:
edades_result = edades_result.rename(columns = {'sección' : 'cod_sec'})

In [38]:
edades_result

,cod_sec,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,...,70-74,75-79,80-84,85-89,90-94,95-99,100 y más,Total,Hombres,Mujeres
0,0100101001,119,103,65,47,45,68,117,215,154,...,30,26,13,4,7,1,0,1341,698,643
1,0100101002,131,120,89,58,59,75,152,201,176,...,32,27,22,10,6,0,0,1528,786,742
2,0100201001,91,60,81,86,104,137,128,118,130,...,68,79,71,42,15,1,0,1979,1012,967
3,0100201002,64,44,52,75,97,110,114,98,105,...,52,70,49,31,9,5,0,1702,894,808
4,0100201003,46,43,41,45,45,80,78,76,69,...,41,31,43,23,12,0,0,1042,517,525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35973,5200108010,74,60,84,80,115,129,120,92,93,...,49,42,37,11,2,0,0,1480,735,745
35974,5200108011,148,155,148,169,162,144,147,140,167,...,33,36,19,13,4,0,0,2107,1057,1050
35975,5200108012,143,145,125,118,129,196,249,215,219,...,75,70,45,30,4,1,0,2532,1307,1225
35976,5200108013,108,84,80,81,150,193,189,165,124,...,33,34,21,2,5,1,0,1762,974,788


Finalmente guardamos este dataset en un fichero, en este caso '2012_df_edades_secciones.csv', y hacemos algo análogo con los datos de 2015, 2016 y 2019, cuando ambas elecciones de ese año comparten las distribuciones de edades y sexos, al ser éstos datos anuales.

In [39]:
edades_result.to_csv('2012_df_edades_secciones.csv', sep = ',', index = False)

In [40]:
#para guardar el archivo en s3:

from botocore.exceptions import ClientError

s3_client = boto3.client(
    's3',
    aws_access_key_id='XXXXX',
    aws_secret_access_key='XXXXXXX',    
)

def upload_file(file_name, bucket, object_name=None):
    """Subir un archivo a un bucket
    :param file_name: archivo que hay que subir
    :param bucket: Bucket al que hay que subirlo
    :param object_name: S3 object name. Incluye la carpeta en la que hay que guardarlo. si no hay no se pone nada
    :return: True si sube el archivo, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    #s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [41]:
upload_file('2012_df_edades_secciones.csv',
            'electomedia',
            object_name = "INE/" + '2012_df_edades_secciones.csv'
           )

True